# Step 8 - Exploratory analysis
## Project: Algorithmic bicycle network growth

This notebook is a sandbox for exploring results.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2021-02-08  
Last modified: 2021-02-08

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

### Constants

In [ ]:
pois = ["grid", "railwaystation"]
measures = ["betweenness", "closeness"]
combs = list(itertools.product(pois, measures))
print(combs)

## Directness and Efficiency for all cities

In [ ]:
# Load results
analysis_result = {}
for p in pois:
    analysis_result[p] = {}
    for m in measures:
        analysis_result[p][m] = {}
        
        for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
            # PLOT Analysis
            filename = placeid + '_poi_' + p + "_" + m + ".csv"
            analysis_result[p][m][placeid] = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
            if len(analysis_result[p][m][placeid]) == 0:
                analysis_result[p][m][placeid] = analysis_result[p][m][list(cities.keys())[0]]
                for n in analysis_result[p][m][placeid].dtype.names:
                    analysis_result[p][m][placeid][n] = [-1]*len(analysis_result[p][m][placeid][n])

### Directness min/max/end

In [ ]:
directness_lcc = {}
for p in pois:
    directness_lcc[p] = {}
    for m in measures:
        directness_lcc[p][m] = {}
        directness_lcc[p][m]["x"] = {}
        directness_lcc[p][m]["y"] = {}

        directness_lcc[p][m]["y"]["min"] = [min(analysis_result[p][m][placeid]["directness_lcc"]) for placeid in cities.keys()]
        directness_lcc[p][m]["y"]["max"] = [max(analysis_result[p][m][placeid]["directness_lcc"]) for placeid in cities.keys()]
        directness_lcc[p][m]["y"]["end"] = [analysis_result[p][m][placeid]["directness_lcc"][-1] for placeid in cities.keys()]

        directness_lcc[p][m]["x"]["min"] = [np.where(analysis_result[p][m][placeid]["directness_lcc"] == min(analysis_result[p][m][placeid]["directness_lcc"]))[0][-1] for placeid in cities.keys()]
        directness_lcc[p][m]["x"]["max"] = [np.where(analysis_result[p][m][placeid]["directness_lcc"] == max(analysis_result[p][m][placeid]["directness_lcc"]))[0][-1] for placeid in cities.keys()]
        directness_lcc[p][m]["x"]["end"] = [np.where(analysis_result[p][m][placeid]["directness_lcc"] == analysis_result[p][m][placeid]["directness_lcc"][-1])[0][-1] for placeid in cities.keys()]       

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (10, 8), squeeze = True)
axes = axes.flatten()

# Plot data
for i,ax in enumerate(axes):
    ind = np.where(np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["min"]) > np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["max"]))[0]
    ax.plot([np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["max"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["min"])[ind]], [np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["max"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["min"])[ind]], ':', color="red", alpha=0.3);
    
#     ind = np.where(np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["min"]) >= 0)[0]
    ax.plot([np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["min"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["end"])[ind]], [np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["min"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["end"])[ind]], ':', color="green", alpha=0.3);
    
    ax.plot(np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["max"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["max"])[ind], '^r', label='max');
    ax.plot(np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["min"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["min"])[ind], 'vg', label='min');
    ax.plot(np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["x"]["end"])[ind], np.asarray(directness_lcc[combs[i][0]][combs[i][1]]["y"]["end"])[ind], 'ok', label='end', markerfacecolor='none');
    

    ax.set_xlabel(combs[i][1] + ' quantile')
    ax.set_ylabel('Directness of LCC')
    ax.set_xlim([-1,40])
    ax.set_ylim([0.4,1]) #0.35
    if i == 0: ax.legend(loc='lower right');
    if i == 1 or i == 3:
        ax.set_ylabel('')
        ax.set_yticklabels([])
        ax.text(42, 0.8, combs[i][0], rotation = 90, horizontalalignment = "center", verticalalignment='center')
    if i == 0 or i == 1:
        ax.set_xlabel('')
        ax.set_xticklabels([])

### Efficiency min/max/end

In [ ]:
efficiency_global = {}
for p in pois:
    efficiency_global[p] = {}
    for m in measures:
        efficiency_global[p][m] = {}
        efficiency_global[p][m]["x"] = {}
        efficiency_global[p][m]["y"] = {}

        efficiency_global[p][m]["y"]["min"] = [min(analysis_result[p][m][placeid]["efficiency_global"]) for placeid in cities.keys()]
        efficiency_global[p][m]["y"]["max"] = [max(analysis_result[p][m][placeid]["efficiency_global"]) for placeid in cities.keys()]
        efficiency_global[p][m]["y"]["end"] = [analysis_result[p][m][placeid]["efficiency_global"][-1] for placeid in cities.keys()]

        efficiency_global[p][m]["x"]["min"] = [np.where(analysis_result[p][m][placeid]["efficiency_global"] == min(analysis_result[p][m][placeid]["efficiency_global"]))[0][-1] for placeid in cities.keys()]
        efficiency_global[p][m]["x"]["max"] = [np.where(analysis_result[p][m][placeid]["efficiency_global"] == max(analysis_result[p][m][placeid]["efficiency_global"]))[0][-1] for placeid in cities.keys()]
        efficiency_global[p][m]["x"]["end"] = [np.where(analysis_result[p][m][placeid]["efficiency_global"] == analysis_result[p][m][placeid]["efficiency_global"][-1])[0][-1] for placeid in cities.keys()]       

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (10, 8), squeeze = True)
axes = axes.flatten()

# Plot data
for i,ax in enumerate(axes):
   
    ind = np.where(np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["min"]) > np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["max"]))[0]
    ax.plot([np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["max"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["min"])[ind]], [np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["max"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["min"])[ind]], ':', color="red", alpha=0.3);
    
#     ind = np.where(np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["min"]) >= 0)[0]
    ax.plot([np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["min"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["end"])[ind]], [np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["min"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["end"])[ind]], ':', color="green", alpha=0.3);
    
    ax.plot(np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["max"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["max"])[ind], '^r', label='max');
    ax.plot(np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["min"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["min"])[ind], 'vg', label='min');
    ax.plot(np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["x"]["end"])[ind], np.asarray(efficiency_global[combs[i][0]][combs[i][1]]["y"]["end"])[ind], 'ok', label='end', markerfacecolor='none');
    

    
    ax.set_xlabel(combs[i][1] + ' quantile')
    ax.set_ylabel('Global Efficiency')
    ax.set_xlim([-1,40])
    ax.set_ylim([0,1])
    if i == 0: ax.legend(loc='lower center');
    if i == 1 or i == 3:
        ax.set_ylabel('')
        ax.set_yticklabels([])
        ax.text(42, 0.55, combs[i][0], rotation = 90, horizontalalignment = "center", verticalalignment='center')
    if i == 0 or i == 1:
        ax.set_xlabel('')
        ax.set_xticklabels([])